In [1]:
# -*- coding=utf-8 -*-
from queue import Queue
from threading import Thread
import threading
import time

from lxml import etree
import requests

import re
url_list = []
actor_list = []
record_movie = []
lock = threading.Lock()
TotalMovie = 6000

class YahooSpider(Thread):
    def __init__(self, q) :
        # 重寫寫父類的__init__方法
        super(YahooSpider, self).__init__()
        self.q = q
        
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36',
        }

    def run(self):
        self.parse_movie()

    def send_request(self, url):
        '''
        用來傳送請求的方法
        :return: 返回網頁原始碼
        '''
        # 請求出錯時，重複請求３次,
        i = 0
        while i <= 3:
            try:
                print
                u"[INFO]請求url:" + url
                html = requests.get(url=url, headers=self.headers).content
                
            except Exception as e:
                print
                u'[INFO] %s%s' % (e, url)
                i += 1
            else:
                return html

    def parse_movie(self):
        '''
        解析網站原始碼，並採用ｘｐａｔｈ提取　電影名稱和平分放到佇列中
        :return:
        '''
        index = 1
        with open('Movie.csv', 'a') as movieFile:
            movieFile.truncate(0)
            while len(record_movie) <= TotalMovie:
                try:
                    url = url_list.pop()
                except IndexError as e:
                    break

                if url in record_movie:
                    continue
                else:
                    record_movie.append(url)
                    list(set(record_movie))
                    lock.acquire()
                    response = self.send_request(url)
                    html = etree.HTML(response)
                    print("Start " + str(index))

                    # 上映日期
                    date = html.xpath('//div[@class="movie_intro_info_r"]/span/text()')[0]

                    # 電影名稱
                    title = html.xpath('//div[@class="movie_intro_info_r"]/h1/text()')[0]

                    # 分類
                    level_name = html.xpath('//div[@class="level_name"]/a/text()')
                    levels = ''
                    if len(level_name) < 1:
                        levels = 'NA '
                    else:
                        for level in level_name:
                            level = level.replace("\n                    ", "")
                            level = level.rstrip()
                            levels += (str(level) + " ")

                    # 劇情介紹
                    r1 = '[a-zA-Z0-9’!"#$%&\'()*+,-./:;<=>?@，。?★、…【】《》？“”‘’！[\\]^_`{|}~]+ '
                    storyText = html.xpath('//span[@id="story"]/text()')
                    Str = ''
                    for story in storyText:
                        story = story.replace("\n                    ", "")
                        story = story.replace("\r\n\xa0\r\n", "")
                        sroty = story.replace("\n", "")
                        story = story.replace("★", "")
                        story = re.sub(r1, '', story)
                        story = story.lstrip()
                        Str += story

                    #演員連結
                    ActorHtml = html.xpath('//a/@href')
                    for Actor in ActorHtml:
                        if 'name_main' in Actor and Actor not in actor_list:
                            actor_list.append(Actor)

                    # 將每一部電影的名稱跟評分加入到佇列
                    AddText = ''
                    AddText = str(title) + " | " + str(levels) + "| " +str(date) + " | " + str(Str.replace("\n", ""))
                    movieFile.write(AddText+"\n")

                    while 1:
                        try:
                            urlAct = actor_list.pop()

                        except IndexError as e:
                            break

                        headers = {
                            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36'
                        }
                        r = requests.get(urlAct, headers=headers)
                        r.encoding = 'utf-8'
                        htmlAct = etree.HTML(r.text)

                        # 電影html
                        MovieHtml = htmlAct.xpath('//a/@href')
                        for Movie in MovieHtml:
                            if 'movieinfo_main' in Movie and Movie not in url_list:
                                url_list.append(Movie)

                    index += 1
                    lock.release()
            
    
class InitUrlSpider(Thread):
    def run(self):
        url = 'https://movies.yahoo.com.tw/movieinfo_main/%E9%BB%91%E5%AF%A1%E5%A9%A6-black-widow-10397' 
        #url = 'https://movies.yahoo.com.tw/movieinfo_main/%E6%BF%80%E6%83%85%E6%97%A5%E6%9C%AC-%E4%BB%8A%E6%9D%91%E6%98%8C%E5%B9%B3%E7%9A%84%E6%98%A0%E7%95%AB%E6%B5%AE%E7%94%9F%E9%8C%84-deadly-passion-japanthe-films-of-shohei-imamura-6312'
        url_list.append(url)
        
        # 構造所有動畫類ｕｒｌ
        base_url = 'https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=12&page='
        base_url2 = 'https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=11&page='
        base_url3 = 'https://movies.yahoo.com.tw/moviegenre_result.html?genre_id=16&page='
        for num in range(1, 46):
            actor_list.append(base_url + str(num))
            actor_list.append(base_url2 + str(num))
            actor_list.append(base_url3 + str(num))
            

def main():
    # 建立一個佇列用來儲存程式獲取到的資料
    qData = Queue()
    a = InitUrlSpider()
    a.start()

    # 儲存執行緒
    Thread_list = []
    # 建立並啟動執行緒
    for i in range(8):
        p = YahooSpider(qData)
        p.start()
        Thread_list.append(p)

    a.join()
    # 讓主執行緒等待子執行緒執行完成
    for i in Thread_list:
        i.join()
        
if __name__ == "__main__":
    start = time.time()
    main()
    print('[info]耗時：%s' % (time.time() - start))
    

Start 1
Start 2
Start 3
Start 4
Start 5
Start 6
Start 7
Start 8
Start 9
Start 10
Start 11
Start 12
Start 13
Start 14
Start 15
Start 16
Start 17
Start 18
Start 19
Start 20
Start 21
Start 22
Start 23
Start 24
Start 25
Start 26
Start 27
Start 28
Start 29
Start 30
Start 31
Start 32
Start 33
Start 34
Start 35
Start 36
Start 37
Start 38
Start 39
Start 40
Start 41
Start 42
Start 43
Start 44
Start 45
Start 46
Start 47
Start 48
Start 49
Start 50
Start 51
Start 52
Start 53
Start 54
Start 55
Start 56
Start 57
Start 58
Start 59
Start 60
Start 61
Start 62
Start 63
Start 64
Start 65
Start 66
Start 67
Start 68
Start 69
Start 70
Start 71
Start 72
Start 73
Start 74
Start 75
Start 76
Start 77
Start 78
Start 79
Start 80
Start 81
Start 82
Start 83
Start 84
Start 85
Start 86
Start 87
Start 88
Start 89
Start 90
Start 91
Start 92
Start 93
Start 94
Start 95
Start 96
Start 97
Start 98
Start 99
Start 100
Start 101
Start 102
Start 103
Start 104
Start 105
Start 106
Start 107
Start 108
Start 109
Start 110
Start 11

Start 832
Start 833
Start 834
Start 835
Start 836
Start 837
Start 838
Start 839
Start 840
Start 841
Start 842
Start 843
Start 844
Start 845
Start 846
Start 847
Start 848
Start 849
Start 850
Start 851
Start 852
Start 853
Start 854
Start 855
Start 856
Start 857
Start 858
Start 859
Start 860
Start 861
Start 862
Start 863
Start 864
Start 865
Start 866
Start 867
Start 868
Start 869
Start 870
Start 871
Start 872
Start 873
Start 874
Start 875
Start 876
Start 877
Start 878
Start 879
Start 880
Start 881
Start 882
Start 883
Start 884
Start 885
Start 886
Start 887
Start 888
Start 889
Start 890
Start 891
Start 892
Start 893
Start 894
Start 895
Start 896
Start 897
Start 898
Start 899
Start 900
Start 901
Start 902
Start 903
Start 904
Start 905
Start 906
Start 907
Start 908
Start 909
Start 910
Start 911
Start 912
Start 913
Start 914
Start 915
Start 916
Start 917
Start 918
Start 919
Start 920
Start 921
Start 922
Start 923
Start 924
Start 925
Start 926
Start 927
Start 928
Start 929
Start 930
Start 931


Start 1593
Start 1594
Start 1595
Start 1596
Start 1597
Start 1598
Start 1599
Start 1600
Start 1601
Start 1602
Start 1603
Start 1604
Start 1605
Start 1606
Start 1607
Start 1608
Start 1609
Start 1610
Start 1611
Start 1612
Start 1613
Start 1614
Start 1615
Start 1616
Start 1617
Start 1618
Start 1619
Start 1620
Start 1621
Start 1622
Start 1623
Start 1624
Start 1625
Start 1626
Start 1627
Start 1628
Start 1629
Start 1630
Start 1631
Start 1632
Start 1633
Start 1634
Start 1635
Start 1636
Start 1637
Start 1638
Start 1639
Start 1640
Start 1641
Start 1642
Start 1643
Start 1644
Start 1645
Start 1646
Start 1647
Start 1648
Start 1649
Start 1650
Start 1651
Start 1652
Start 1653
Start 1654
Start 1655
Start 1656
Start 1657
Start 1658
Start 1659
Start 1660
Start 1661
Start 1662
Start 1663
Start 1664
Start 1665
Start 1666
Start 1667
Start 1668
Start 1669
Start 1670
Start 1671
Start 1672
Start 1673
Start 1674
Start 1675
Start 1676
Start 1677
Start 1678
Start 1679
Start 1680
Start 1681
Start 1682
Start 1683

Start 2338
Start 2339
Start 2340
Start 2341
Start 2342
Start 2343
Start 2344
Start 2345
Start 2346
Start 2347
Start 2348
Start 2349
Start 2350
Start 2351
Start 2352
Start 2353
Start 2354
Start 2355
Start 2356
Start 2357
Start 2358
Start 2359
Start 2360
Start 2361
Start 2362
Start 2363
Start 2364
Start 2365
Start 2366
Start 2367
Start 2368
Start 2369
Start 2370
Start 2371
Start 2372
Start 2373
Start 2374
Start 2375
Start 2376
Start 2377
Start 2378
Start 2379
Start 2380
Start 2381
Start 2382
Start 2383
Start 2384
Start 2385
Start 2386
Start 2387
Start 2388
Start 2389
Start 2390
Start 2391
Start 2392
Start 2393
Start 2394
Start 2395
Start 2396
Start 2397
Start 2398
Start 2399
Start 2400
Start 2401
Start 2402
Start 2403
Start 2404
Start 2405
Start 2406
Start 2407
Start 2408
Start 2409
Start 2410
Start 2411
Start 2412
Start 2413
Start 2414
Start 2415
Start 2416
Start 2417
Start 2418
Start 2419
Start 2420
Start 2421
Start 2422
Start 2423
Start 2424
Start 2425
Start 2426
Start 2427
Start 2428

Start 3083
Start 3084
Start 3085
Start 3086
Start 3087
Start 3088
Start 3089
Start 3090
Start 3091
Start 3092
Start 3093
Start 3094
Start 3095
Start 3096
Start 3097
Start 3098
Start 3099
Start 3100
Start 3101
Start 3102
Start 3103
Start 3104
Start 3105
Start 3106
Start 3107
Start 3108
Start 3109
Start 3110
Start 3111
Start 3112
Start 3113
Start 3114
Start 3115
Start 3116
Start 3117
Start 3118
Start 3119
Start 3120
Start 3121
Start 3122
Start 3123
Start 3124
Start 3125
Start 3126
Start 3127
Start 3128
Start 3129
Start 3130
Start 3131
Start 3132
Start 3133
Start 3134
Start 3135
Start 3136
Start 3137
Start 3138
Start 3139
Start 3140
Start 3141
Start 3142
Start 3143
Start 3144
Start 3145
Start 3146
Start 3147
Start 3148
Start 3149
Start 3150
Start 3151
Start 3152
Start 3153
Start 3154
Start 3155
Start 3156
Start 3157
Start 3158
Start 3159
Start 3160
Start 3161
Start 3162
Start 3163
Start 3164
Start 3165
Start 3166
Start 3167
Start 3168
Start 3169
Start 3170
Start 3171
Start 3172
Start 3173

Start 3828
Start 3829
Start 3830
Start 3831
Start 3832
Start 3833
Start 3834
Start 3835
Start 3836
Start 3837
Start 3838
Start 3839
Start 3840
Start 3841
Start 3842
Start 3843
Start 3844
Start 3845
Start 3846
Start 3847
Start 3848
Start 3849
Start 3850
Start 3851
Start 3852
Start 3853
Start 3854
Start 3855
Start 3856
Start 3857
Start 3858
Start 3859
Start 3860
Start 3861
Start 3862
Start 3863
Start 3864
Start 3865
Start 3866
Start 3867
Start 3868
Start 3869
Start 3870
Start 3871
Start 3872
Start 3873
Start 3874
Start 3875
Start 3876
Start 3877
Start 3878
Start 3879
Start 3880
Start 3881
Start 3882
Start 3883
Start 3884
Start 3885
Start 3886
Start 3887
Start 3888
Start 3889
Start 3890
Start 3891
Start 3892
Start 3893
Start 3894
Start 3895
Start 3896
Start 3897
Start 3898
Start 3899
Start 3900
Start 3901
Start 3902
Start 3903
Start 3904
Start 3905
Start 3906
Start 3907
Start 3908
Start 3909
Start 3910
Start 3911
Start 3912
Start 3913
Start 3914
Start 3915
Start 3916
Start 3917
Start 3918

Start 4573
Start 4574
Start 4575
Start 4576
Start 4577
Start 4578
Start 4579
Start 4580
Start 4581
Start 4582
Start 4583
Start 4584
Start 4585
Start 4586
Start 4587
Start 4588
Start 4589
Start 4590
Start 4591
Start 4592
Start 4593
Start 4594
Start 4595
Start 4596
Start 4597
Start 4598
Start 4599
Start 4600
Start 4601
Start 4602
Start 4603
Start 4604
Start 4605
Start 4606
Start 4607
Start 4608
Start 4609
Start 4610
Start 4611
Start 4612
Start 4613
Start 4614
Start 4615
Start 4616
Start 4617
Start 4618
Start 4619
Start 4620
Start 4621
Start 4622
Start 4623
Start 4624
Start 4625
Start 4626
Start 4627
Start 4628
Start 4629
Start 4630
Start 4631
Start 4632
Start 4633
Start 4634
Start 4635
Start 4636
Start 4637
Start 4638
Start 4639
Start 4640
Start 4641
Start 4642
Start 4643
Start 4644
Start 4645
Start 4646
Start 4647
Start 4648
Start 4649
Start 4650
Start 4651
Start 4652
Start 4653
Start 4654
Start 4655
Start 4656
Start 4657
Start 4658
Start 4659
Start 4660
Start 4661
Start 4662
Start 4663

Start 5318
Start 5319
Start 5320
Start 5321
Start 5322
Start 5323
Start 5324
Start 5325
Start 5326
Start 5327
Start 5328
Start 5329
Start 5330
Start 5331
Start 5332
Start 5333
Start 5334
Start 5335
Start 5336
Start 5337
Start 5338
Start 5339
Start 5340
Start 5341
Start 5342
Start 5343
Start 5344
Start 5345
Start 5346
Start 5347
Start 5348
Start 5349
Start 5350
Start 5351
Start 5352
Start 5353
Start 5354
Start 5355
Start 5356
Start 5357
Start 5358
Start 5359
Start 5360
Start 5361
Start 5362
Start 5363
Start 5364
Start 5365
Start 5366
Start 5367
Start 5368
Start 5369
Start 5370
Start 5371
Start 5372
Start 5373
Start 5374
Start 5375
Start 5376
Start 5377
Start 5378
Start 5379
Start 5380
Start 5381
Start 5382
Start 5383
Start 5384
Start 5385
Start 5386
Start 5387
Start 5388
Start 5389
Start 5390
Start 5391
Start 5392
Start 5393
Start 5394
Start 5395
Start 5396
Start 5397
Start 5398
Start 5399
Start 5400
Start 5401
Start 5402
Start 5403
Start 5404
Start 5405
Start 5406
Start 5407
Start 5408